In [1]:
import numpy as np
from mgwr.sel_bw import Sel_BW
from mgwr.gwr import GWR, MGWR
import pandas as pd
import libpysal as ps
from numpy.linalg import matrix_rank



In [2]:
d = ps.io.open(ps.examples.get_path('GData_utm.csv'))


langs = ['pop', 'dusaf', 'rete_fer', 'autostrad',
                                 'strade_pri', 'strade_sec', 'h_mean','st_ferrov',
                                 'st_metro','int_prim','int_prim_s','int_sec','scn','dsf11',
                                 'dsf12','dsf13','dsf14','dsf2','dsf3','dsf4','dsf5']

data = pd.read_csv('table.csv')



df  = pd.DataFrame(data, columns = langs)

X = df.to_numpy()
print(matrix_rank(X))
lat = pd.DataFrame(data, columns= ['lat'])
lat = lat['lat'].tolist()

lon = pd.DataFrame(data, columns= ['lon'])
lon = lon['lon'].tolist()


coords = list(zip(lat, lon))

y1 = pd.DataFrame(data, columns= ['FBpop_tot']).to_numpy()
y2 = pd.DataFrame(data, columns= ['mov_tot']).to_numpy()
#coords = list(zip(data.by_col('X'), data.by_col('Y')))
#y = np.array(data.by_col('PctBach')).reshape((-1,1))
#rural  = np.array(data.by_col('PctRural')).reshape((-1,1))
#pov = np.array(data.by_col('PctPov')).reshape((-1,1))
#black = np.array(data.by_col('PctBlack')).reshape((-1,1))
#fb = np.array(data.by_col('PctFB')).reshape((-1,1))
#pop = np.array(data.by_col('TotPop90')).reshape((-1,1))
#X = np.hstack([fb, black, rural])



21


In [3]:
X = (X - X.mean(axis=0)) / X.std(axis=0)


y2 = y2.reshape((-1,1))

y2 = (y2 - y2.mean(axis=0)) / y2.std(axis=0)

sel = Sel_BW(coords, y2, X)

bw = sel.search()
print('bw:', bw)
gwr = GWR(coords, y2, X, bw)
gwr_results = gwr.fit()
print('aicc:', gwr_results.aicc)
print('ENP:', gwr_results.ENP)
print('sigma2:', gwr_results.sigma2)

/Users/Brescia/opt/anaconda3/envs/notebooks/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=1.56749e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/Brescia/opt/anaconda3/envs/notebooks/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=3.72448e-19): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/Brescia/opt/anaconda3/envs/notebooks/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=9.24023e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/Brescia/opt/anaconda3/envs/notebooks/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=7.13896e-20): result may not be accurate.
  xtx_inv_xt = linalg.solve(xtx, xT)
/Users/Brescia/opt/anaconda3/envs/notebooks/lib/python3.9/site-packages/spglm/iwls.py:37: LinAlgWarning: Ill-conditioned matrix (rcond=2.07125e-

bw: 131.0
aicc: -786.9435423720731
ENP: 165.83524908112386
sigma2: 0.010540040116465014


In [4]:
#X = (X - X.mean(axis=0)) / X.std(axis=0)

#y1 = y1.reshape((-1,1))

#y1 = (y1 - y1.mean(axis=0)) / y1.std(axis=0)

selector = Sel_BW(coords, y2, X, multi=True, constant=True)
bw = selector.search(multi_bw_min=[2], multi_bw_max=[159])



Backfitting:   0%|          | 0/200 [00:00<?, ?it/s]

In [5]:
print("bw( intercept ):", bw[0])

print('\n')
for i in range(len(langs)):
    print("bw(",langs[i], '): ', bw[i+1])
print('\n')

mgwr = MGWR(coords, y2, X, selector, constant=True)
mgwr_results = mgwr.fit()
print('aicc:', mgwr_results.aicc)
print('sigma2:', mgwr_results.sigma2)
print('ENP(model):', mgwr_results.ENP)
print('adj_alpha(model):', mgwr_results.adj_alpha[1])
print('critical_t(model):', mgwr_results.critical_tval(alpha=mgwr_results.adj_alpha[1]))
alphas = mgwr_results.adj_alpha_j[:,1]
critical_ts = mgwr_results.critical_tval()
print('\n')
print('ENP(intercept):', mgwr_results.ENP_j[0])
print('adj_alpha(intercept):', alphas[0])
print('critical_t(intercept):', critical_ts[0])
print('\n')
for i in range(len(langs)):
    print("ENP(",langs[i], '): ', mgwr_results.ENP_j[i+1])
    print("adj_alpha(",langs[i], '): ', alphas[i+1])
    print("critical_t(",langs[i], '): ', critical_ts[i+1])


bw( intercept ): 76.0


bw( pop ):  83.0
bw( dusaf ):  156.0
bw( rete_fer ):  102.0
bw( autostrad ):  53.0
bw( strade_pri ):  102.0
bw( strade_sec ):  148.0
bw( h_mean ):  25.0
bw( st_ferrov ):  158.0
bw( st_metro ):  39.0
bw( int_prim ):  145.0
bw( int_prim_s ):  65.0
bw( int_sec ):  154.0
bw( scn ):  158.0
bw( dsf11 ):  158.0
bw( dsf12 ):  127.0
bw( dsf13 ):  76.0
bw( dsf14 ):  76.0
bw( dsf2 ):  25.0
bw( dsf3 ):  158.0
bw( dsf4 ):  157.0
bw( dsf5 ):  122.0




Inference:   0%|          | 0/1 [00:00<?, ?it/s]

aicc: 26677.298124741872
sigma2: 207032684976.37845
ENP(model): -9497965295.260176
adj_alpha(model): -1.158142787223006e-10
critical_t(model): 6.558193618871948


ENP(intercept): 23163841206.415047
adj_alpha(intercept): 2.158536641416489e-12
critical_t(intercept): 7.170582504240732


ENP( pop ):  8075431373.467864
adj_alpha( pop ):  6.1916197027290575e-12
critical_t( pop ):  7.012807894893175
ENP( dusaf ):  1023643522.7002678
adj_alpha( dusaf ):  4.884512908175795e-11
critical_t( dusaf ):  6.694836367852389
ENP( rete_fer ):  -557487196.3025738
adj_alpha( rete_fer ):  -8.968815845747732e-11
critical_t( rete_fer ):  6.598899237176328
ENP( autostrad ):  -19042715466.94631
adj_alpha( autostrad ):  -2.6256759487263404e-12
critical_t( autostrad ):  7.141462615644468
ENP( strade_pri ):  2526937427.39243
adj_alpha( strade_pri ):  1.978679782807106e-11
critical_t( strade_pri ):  6.835440739833636
ENP( strade_sec ):  -649312547.6574535
adj_alpha( strade_sec ):  -7.700451836390143e-11
critical_t(

In [6]:
mgwr_results.summary()


Model type                                                         Gaussian
Number of observations:                                                 614
Number of covariates:                                                    22

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                             14.802
Log-likelihood:                                                     272.422
AIC:                                                               -500.844
AICc:                                                              -496.972
BIC:                                                              -3785.835
R2:                                                                   0.976
Adj. R2:                                                              0.975

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------